In [2]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

2023-11-08 11:42:13.726977: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 11:42:13.765429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 11:42:13.765457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 11:42:13.765474: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 11:42:13.771617: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 11:42:13.772299: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Using TensorFlow backend


In [3]:
adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [4]:
adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [15]:
def build_model(hp):
    model = keras.Sequential()
    #model.add(keras.layers.Dense(units=8000, activation="relu"))
    if hp.Boolean("lstm"):
        #model.add(keras.layers.Dropout(rate=0.1))
        model.add(keras.layers.Reshape((adv_X.shape[1],1)))
        model.add(keras.layers.LSTM(hp.Int('units_lstm', min_value=2, max_value=30, step=4), activation='relu', return_sequences=False, return_state=False))
    else:
        model.add(keras.layers.Dense(units=hp.Int('units_dense1', min_value=8, max_value=64, step=8)))
    model.add(keras.layers.Dense(units=hp.Int('units_dense2', min_value=4, max_value=32, step=4)))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [16]:
!rm -rf ./untitled_project

In [17]:
from keras_tuner import Objective
tuner = RandomSearch(
    build_model,
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = Objective(name="val_r2_score", direction="max"),
    max_trials = 16,
    executions_per_trial = 4,
    tuner_id="08.11.adversary"
)

In [18]:
tuner.search(adv_X, adv_y, epochs=10, batch_size=64, validation_data=(adv_X_test, adv_y_test))

Trial 16 Complete [00h 39m 43s]
val_r2_score: -0.0546116828918457

Best val_r2_score So Far: 0.0013479441404342651
Total elapsed time: 04h 48m 41s


In [19]:
models = tuner.get_best_models(num_models=2)

In [20]:
best_model = models[0]

In [21]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [22]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 10001, 1)          0         
                                                                 
 lstm (LSTM)                 (None, 2)                 32        
                                                                 
 dense (Dense)               (None, 20)                60        
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)